In [ ]:
## Sage - Smart's Attack - https://hxp.io/blog/25/SharifCTF-2016-crypto350-British-Elevator-writeup/
p = 0xa15c4fb663a578d8b2496d3151a946119ee42695e18e13e90600192b1d0abdbb6f787f90c8d102ff88e284dd4526f5f6b6c980bf88f1d0490714b67e8a2a2b77
A = 0x5e009506fcc7eff573bc960d88638fe25e76a9b6c7caeea072a27dcd1fa46abb15b7b6210cf90caba982893ee2779669bac06e267013486b22ff3e24abae2d42
B = 0x2ce7d1ca4493b0977f088f6d30d9241f8048fdea112cc385b793bce953998caae680864a7d3aa437ea3ffd1441ca3fb352b0b710bb3f053e980e503be9a7fece
E = EllipticCurve(GF(p), [0, 0, 0, A, B])
E.order() == p

xP = 3034712809375537908102988750113382444008758539448972750581525810900634243392172703684905257490982543775233630011707375189041302436945106395617312498769005
yP = 4986645098582616415690074082237817624424333339074969364527548107042876175480894132576399611027847402879885574130125050842710052291870268101817275410204850
xQ = 4748198372895404866752111766626421927481971519483471383813044005699388317650395315193922226704604937454742608233124831870493636003725200307683939875286865
yQ = 2421873309002279841021791369884483308051497215798017509805302041102468310636822060707350789776065212606890489706597369526562336256272258544226688832663757

E = EllipticCurve(GF(p), [0, 0, 0, A, B])
assert E.order() == p

Qp = Qp(p, 2)
Ep = EllipticCurve(Qp, [0, 0, 0, A, B])

yPp = sqrt(Qp(xP) ** 3 + Qp(A) * Qp(xP) + Qp(B))
Pp = Ep(Qp(xP), (-yPp, yPp)[yPp[0] == yP])

yQp = sqrt(Qp(xQ) ** 3 + Qp(A) * Qp(xQ) + Qp(B))
Qp = Ep(Qp(xQ), (-yQp, yQp)[yQp[0] == yQ])

print('Pp = {}'.format(Pp))
print('Qp = {}'.format(Qp))
print('-' * 40)

lQ = Ep.formal_group().log()(- (p * Qp)[0] // (p * Qp)[1]) / p
print('log(Q) = {}'.format(lQ))
lP = Ep.formal_group().log()(- (p * Pp)[0] // (p * Pp)[1]) / p
print('log(P) = {}'.format(lP))
print('-' * 40)

e = lQ / lP
print('e = {}'.format(e))

assert e[0] * E(xP, yP) == E(xQ, yQ)

print('\n--> FLAG: {:d}\n'.format(e[0]))

In [1]:
p = 404993569381
P = (391109997465, 167359562362)
nP = (209038982304, 168517698208)
b = 54575449882

In [2]:
x = 391109997465
y = 167359562362
from Crypto.Util.number import inverse

In [3]:
a = ((pow(y, 2, p) - pow(x, 3, p) - b)%p * inverse(x, p))%p

In [4]:
a

316508952642

In [ ]:
p = 404993569381
P = (391109997465, 167359562362)
nP = (209038982304, 168517698208)
b = 54575449882
a = 316508952642

## Sage - MOV Attack
M = p
A = a
B = b

F = GF(M)
E = EllipticCurve(F, [A,B])

P = E(P)
nP = E(nP)

print(discrete_log(nP,P,P.order(),operation='+'))
# flag{17683067357}

# SMol E

In [1]:
N = 163741039289512913448211316444208415089696281156598707546239939060930005300801050041110593445808590019811244791595198691653105173667082682192119631702680644123546329907362913533410257711393278981293987091294252121612050351292239086354120710656815218407878832422193841935690159084860401941224426397820742950923

E = 3

C1 = 110524539798470366613834133888472781069399552085868942087632499354651575111511036068021885688092481936060366815322764760005015342876190750877958695168393505027738910101191528175868547818851667359542590042073677436170569507102025782872063324950368166532649021589734367946954269468844281238141036170008727208883

C2 = 42406837735093367941682857892181550522346220427504754988544140886997339709785380303682471368168102002682892652577294324286913907635616629790484019421641636805493203989143298536257296680179745122126655008200829607192191208919525797616523271426092158734972067387818678258432674493723618035248340048171787246777

In [ ]:
#!/usr/bin/sage
#!encoding: UTF-8
#
# Solution for the rsa1 challenge of CONFidence CTF 2015.
# Uses Coppersmith's short padding attack and the Franklin-Reiter attack.
#
# See http://www.di.ens.fr/~fouque/ens-rennes/coppersmith.pdf
# and https://www.cs.unc.edu/~reiter/papers/1996/Eurocrypt.pdf

# Copyright (c) 2015 Samuel Groß
#

from binascii import unhexlify

def coppersmith_short_pad(c1, c2, n):
    """When provided with two ciphertexts of the same message but with different (short)
    paddings, i.e.
        C1 = (M | r1)^3 = M1^3 and
        C2 = (M | r2)^3 = M2^3),
    this function calculates the difference r2 - r1 of the two (random) paddings used
    during encryption using Coppersmith's method. Afterwards, the Franklin-Reiter
    attack can be applied to recover the plaintext.

    See http://www.di.ens.fr/~fouque/ens-rennes/coppersmith.pdf

    Returns all possible values for the difference in a list.
    """
    # Fixed exponent.
    e = 3

    # Create two polynomial rings and corresponding variables. Sage cannot calculate the resultant
    # in a ring of integers modulo N, so we'll use and integer ring for that, then switch rings afterwards.
    RZmodN.<xm> = PolynomialRing(Zmod(n))
    RZ.<x, y> = PolynomialRing(ZZ)

    # Create the two polynomials g1 and g2. If y = r2 - r1, then g1 and g2 both have the root M1.
    g1 = x**e - c1
    g2 = (x + y)**e - c2

    # Calculate the resultant. The resultant is the product of the differences of the roots of the
    # two polynomials. As seen above, for the case that y = r2 - r1, both polynomials have the same
    # root M1, thus r2 - r1 is a root of the resultant of g1 and g2.
    p = g1.resultant(g2)

    #
    # Now let sage do all the work by calculating small roots of the resultant.
    #

    # We need a univariate polynomial for this. At this point, x should be eliminated though.
    p = p.univariate_polynomial()
    # Change rings. We need the ring of integers modulo N now.
    p = p.change_ring(RZmodN).subs(y = xm)
    # Make sure the polynomial is monic, i.e. the coefficient of x^(p.degree()) is 1.
    p = p.monic()
    # Now let sage to the magic. Try to find all small roots of p using Coppersmith's method.
    return p.small_roots(X=2**(512 // 9), beta=0.5)

def franklin_reiter(c1, c2, n, a=1, b=1):
    """Recovers the plain text message from the two cipher texts knowing
    that they were both encrypted using the exponent 3 and that
    M2 = a * M1 + b.

    See https://www.cs.unc.edu/~reiter/papers/1996/Eurocrypt.pdf

    Returns both plain text messages as a tuple.
    """
    f = b*(c2 + 2*(a**3)*c1 - b**3  ) % n
    g = a*(c2 - a**3*c1     + 2*b**3) % n

    # Calculage f / g: f / g = f * g**(-1) = f * inverse_mod(g, n)
    gi = inverse_mod(g, n)
    m = f * gi % n

    return (m, m+b)

def decode(m):
    """Returns the provided number interpreted as a string."""
    h = hex(m)

    if h.startswith('0x'):
        h = h[2:]

    if len(h) % 2 != 0:
        h = '0' + h

    return unhexlify(h)


def main():
    # Input: Ciphertexts C1, C2, with
    #   C1 = (M | r1)^3 = M1^3
    #   C2 = (M | r2)^3 = M2^3
    N = 163741039289512913448211316444208415089696281156598707546239939060930005300801050041110593445808590019811244791595198691653105173667082682192119631702680644123546329907362913533410257711393278981293987091294252121612050351292239086354120710656815218407878832422193841935690159084860401941224426397820742950923
    # flag0
    C1 = 110524539798470366613834133888472781069399552085868942087632499354651575111511036068021885688092481936060366815322764760005015342876190750877958695168393505027738910101191528175868547818851667359542590042073677436170569507102025782872063324950368166532649021589734367946954269468844281238141036170008727208883
    # flag1
    C2 = 42406837735093367941682857892181550522346220427504754988544140886997339709785380303682471368168102002682892652577294324286913907635616629790484019421641636805493203989143298536257296680179745122126655008200829607192191208919525797616523271426092158734972067387818678258432674493723618035248340048171787246777

    #
    # Step 1: Use the Coppersmith Short Padding attack to recover the difference (r2 - r1)
    # between the two paddings.
    #
    roots = coppersmith_short_pad(C1, C2, N)
    print(roots)
    assert(roots)

    # The root can be negative (in which case we get a huge root, since we're still in the ring of integers modulo N),
    # because r2 might be smaller than r1. In this case invert the root and switch C1 and C2.
    #
    # TODO check if we got multiple roots. In that case to Franklin-Reiter for each of them
    # and check if the recovered message encrypts to the ciphertext again.
    # (Or just check for ASCII or known words)
    d = roots[0]
#     if d >= 2**32:
#         print("hi")
#         C1, C2 = C2, C1
#         d = -d

    #
    # Step 2: Use the Franklin-Reiter Attack to recover the plaintext from the ciphertexts.
    # The Franklin-Reiter Attack is applicable if e = 3 and M2 = a * M1 + b.
    # We know b from step 1, and a = 1.
    #
    m, n = franklin_reiter(C1, C2, N, b=int(d))

    # Remove the padding. We know its 32 bit long.
#     m = m >> 32

    # All done :)
    print(decode(m))
    print(m)
    print(n)

if __name__ == '__main__':
    main()

In [2]:
m = 1426051161596273413795556654328320105145439332147585418507576775870780450590379567453641429082640842935901398525237698534587016076610446383728128936582478631369081375319103785503713430762835018940932512662482247881629813321166872870577809910090459052486979919351415977663885575951
n = 163741039289512913448211316445634466251292554570394264200568259166075444632948635459618170221679370470401624359048840120735746016602984080717357330237267660200156776291091042469992736342762360356613090876797965552374885370233171599016602958538445031729045705292771651845780618137347381860575839437540610020083

In [3]:
n1 = n%N

In [4]:
n1

1426051161596273413795556654328320105145439332147585418507576775870780450590379567453641429082640842935901398525237698534587016076610446383728128936582478631369081375319103785503713430762835018940932512662482247881629813321166872870577809910090459052486979919351413039719867069160

In [5]:
pow(n1, 3, N) == C1

True

In [6]:
pow(m, 3, N) == C2

True

In [7]:
from Crypto.Util.number import long_to_bytes, bytes_to_long

In [8]:
a = long_to_bytes(n1)
b = long_to_bytes(m)

In [9]:
n1

1426051161596273413795556654328320105145439332147585418507576775870780450590379567453641429082640842935901398525237698534587016076610446383728128936582478631369081375319103785503713430762835018940932512662482247881629813321166872870577809910090459052486979919351413039719867069160

In [10]:
a

b'\xa0\xe4\xca\xe6\xe6@\xa0\xde\xd2\xdc\xe8@\x8c@\xe8\xde@\xe0\xc2\xf2@\xe4\xca\xe6\xe0\xca\xc6\xe8\xe6\\@\x92N\xda@\xee\xe4\xd2\xe8\xd2\xdc\xce@\xe8\xd0\xd2\xe6@\xc2@\xc8\xc2\xf2@\xc4\xca\xcc\xde\xe4\xca@\x90\xa6\x86\xa8\x8c@\xe6\xe8\xc2\xe4\xe8\xe6\\@\xcc\xd8\xc2\xce\xf6\xdc`\xe8\xbeh\xbe\xecf\xe4\xf2\xbe\xe6\xda`\xd8\xbe\xcc\xd8h\xce\xfa\x14\x00\x00\x00\x00\x00\x00\x00\x01\xf8\x96\xaaI\xb2\xe8'

In [11]:
b

b'\xa0\xe4\xca\xe6\xe6@\xa0\xde\xd2\xdc\xe8@\x8c@\xe8\xde@\xe0\xc2\xf2@\xe4\xca\xe6\xe0\xca\xc6\xe8\xe6\\@\x92N\xda@\xee\xe4\xd2\xe8\xd2\xdc\xce@\xe8\xd0\xd2\xe6@\xc2@\xc8\xc2\xf2@\xc4\xca\xcc\xde\xe4\xca@\x90\xa6\x86\xa8\x8c@\xe6\xe8\xc2\xe4\xe8\xe6\\@\xcc\xd8\xc2\xce\xf6\xdc`\xe8\xbeh\xbe\xecf\xe4\xf2\xbe\xe6\xda`\xd8\xbe\xcc\xd8h\xce\xfa\x14\x00\x00\x00\x00\x00\x00\x00\x0ch\xa2\x0f\xe1\x87\x0f'

In [12]:
a.hex()

'a0e4cae6e640a0ded2dce8408c40e8de40e0c2f240e4cae6e0cac6e8e65c40924eda40eee4d2e8d2dcce40e8d0d2e640c240c8c2f240c4caccdee4ca4090a686a88c40e6e8c2e4e8e65c40ccd8c2cef6dc60e8be68beec66e4f2bee6da60d8beccd868cefa140000000000000001f896aa49b2e8'

In [13]:
b.hex()

'a0e4cae6e640a0ded2dce8408c40e8de40e0c2f240e4cae6e0cac6e8e65c40924eda40eee4d2e8d2dcce40e8d0d2e640c240c8c2f240c4caccdee4ca4090a686a88c40e6e8c2e4e8e65c40ccd8c2cef6dc60e8be68beec66e4f2bee6da60d8beccd868cefa14000000000000000c68a20fe1870f'

In [14]:
flag = b''
cnt = 0
for i in range(len(a)):
    if(a[i] == b[i]):
        flag += bytes([a[i]])
    else:
        cnt += 1
print(cnt)

7


In [15]:
f = bytes_to_long(flag)
f

19790435423720406840569172789550124529698295785356895994216663512848825714906802578830233700394337122639729066936843650739527801182971544543100619003884713556066806895970874656583335809462640116642325081343121271926635736398509724945052461326933921532774950371328

In [19]:
pre = int("a0e4cae6e640a0ded2dce8408c40e8de40e0c2f240e4cae6e0cac6e8e65c40924eda40eee4d2e8d2dcce40e8d0d2e640c240c8c2f240c4caccdee4ca4090a686a88c40e6e8c2e4e8e65c40ccd8c2cef6dc60e8be68beec66e4f2bee6da60d8beccd868cefa14", 16)

In [20]:
hex(pre*16**28)

'0xa0e4cae6e640a0ded2dce8408c40e8de40e0c2f240e4cae6e0cac6e8e65c40924eda40eee4d2e8d2dcce40e8d0d2e640c240c8c2f240c4caccdee4ca4090a686a88c40e6e8c2e4e8e65c40ccd8c2cef6dc60e8be68beec66e4f2bee6da60d8beccd868cefa140000000000000000000000000000'

In [25]:
2**112 * pre + 0x01f896aa49b2e8 == n1

True

In [26]:
bitl = N.bit_length()

In [28]:
bitl//9

113

In [29]:
pre

274647463434646394450216644177871834898972622026570800022003175822738276142070992497521332581432955725061055104142033338736725317271885401280992741788809973714030705726636331317815573192268054312258422374514195029119253083062370326198515531708948

In [30]:
pre//2

137323731717323197225108322088935917449486311013285400011001587911369138071035496248760666290716477862530527552071016669368362658635942700640496370894404986857015352863318165658907786596134027156129211187257097514559626541531185163099257765854474

In [31]:
2**113 * 137323731717323197225108322088935917449486311013285400011001587911369138071035496248760666290716477862530527552071016669368362658635942700640496370894404986857015352863318165658907786596134027156129211187257097514559626541531185163099257765854474 + 0x01f896aa49b2e8 == n1

True

In [32]:
long_to_bytes(137323731717323197225108322088935917449486311013285400011001587911369138071035496248760666290716477862530527552071016669368362658635942700640496370894404986857015352863318165658907786596134027156129211187257097514559626541531185163099257765854474)

b"Press Point F to pay respects. I'm writing this a day before HSCTF starts. flag{n0t_4_v3ry_sm0l_fl4g}\n"

# Morbid

In [94]:
m = {}
m[9] = "xx"
m[8] = "x-"
m[7] = "x."
m[6] = "-x"
m[5] = "--"
m[4] = "-."
m[3] = ".x"
m[2] = ".-"
m[1] = ".."

In [95]:
import itertools
x = list(itertools.permutations([1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [103]:
MORSE_CODE_DICT = { 'A':'.-', 'B':'-...', 
                    'C':'-.-.', 'D':'-..', 'E':'.', 
                    'F':'..-.', 'G':'--.', 'H':'....', 
                    'I':'..', 'J':'.---', 'K':'-.-', 
                    'L':'.-..', 'M':'--', 'N':'-.', 
                    'O':'---', 'P':'.--.', 'Q':'--.-', 
                    'R':'.-.', 'S':'...', 'T':'-', 
                    'U':'..-', 'V':'...-', 'W':'.--', 
                    'X':'-..-', 'Y':'-.--', 'Z':'--..', 
                    '1':'.----', '2':'..---', '3':'...--', 
                    '4':'....-', '5':'.....', '6':'-....', 
                    '7':'--...', '8':'---..', '9':'----.', 
                    '0':'-----', ', ':'--..--', '.':'.-.-.-', 
                    '?':'..--..', '/':'-..-.', '-':'-....-', 
                    '(':'-.--.', ')':'-.--.-', '_': '..--.-', '!': '-.-.--', '&': '.-...', '+':'.-.-.', '@': '.--.-.', '=': '-...-',
                      ';': '-.-.-.', ':': '---...', "'": '.----.', 'qopen': '.-..-.', 'qclose': '.-..-.'} 
  
# Function to encrypt the string 
# according to the morse code chart 
def encrypt(message): 
    cipher = '' 
    for letter in message: 
        if letter != ' ': 
  
            # Looks up the dictionary and adds the 
            # correspponding morse code 
            # along with a space to separate 
            # morse codes for different characters 
            cipher += MORSE_CODE_DICT[letter] + ' '
        else: 
            # 1 space indicates different characters 
            # and 2 indicates different words 
            cipher += ' '
  
    return cipher 

import re
# Function to decrypt the string 
# from morse to english 
def decrypt(message): 
    message = message.replace("xx", " ")
    message = message.replace("x", " ")
    message = re.sub('\s+', ' ', message)
    message = message.strip()
    # extra space added at the end to access the 
    # last morse code 
    message += ' '
  
    decipher = '' 
    citext = '' 
    for letter in message: 
  
        # checks for space 
        if (letter != ' '): 
  
            # counter to keep track of space 
            i = 0
  
            # storing morse code of a single character 
            citext += letter 
  
        # in case of space 
        else: 
            # if i = 1 that indicates a new character 
            i += 1
  
            # if i = 2 that indicates a new word 
            if i == 2 : 
  
                 # adding space to separate words 
                decipher += ' '
            else: 
  
                # accessing the keys using their values (reverse of encryption) 
                decipher += list(MORSE_CODE_DICT.keys())[list(MORSE_CODE_DICT 
                .values()).index(citext)] 
                citext = '' 
  
    return decipher 

In [104]:
decrypt("---x-.x-.-.x.xx..-x.--.x---x-.")

'ONCEUPON'

In [105]:
cipher = "118289293938434193849271464117429364476994241473157664969879696938145689474393647294392739247721652822414624317164228466"

In [106]:
# cipher = "27435881512827465679378"

In [107]:
import sys
for comb in x:
    try:
        morse = ''
        for c in cipher:
            morse += m[comb.index(int(c))+1]
        print(decrypt(morse))
        print(morse)
        print()
    except ValueError:
        pass
# flag{M0R3_B1T5}

0KEDSUSPUINMET1LITEENERWE_TTINTENNUMUMEEHITEDHNSNEY)CWEG3MT.ETT
-----x-.-x.x-..x...x..-xx...x.--.x..-xx..x-.xx--x.x-x.----xxx.-..x..x-x.x.xxx-.x.xx.-.x.--x.xx..--.-xxx-x-x..xx-.x-xxx.xx-.xx-.x..-x--x..-x--x.xx.xxx....x..x-x.xx-..xx....x-.xx...x-.x.xxxx-.--x-.--.-x-.-.x.--x.x--.x...--xx--x-x.-.-.-xx.x-x-

TOPIR1WN1ERMTTQIATTNEETAMX/TEE8EEJMIGTKRATNITRqopenACETIWEAMAAGTTEJ
x-x---x.--.xx..x.-.x.----x.--xx-.x.----x.xx.-.x--xxx-xx-x--.-xx..x.-xx-x-x-.xx.x.x-xx.-xx--x-..-x-..-.xxxx-x.xxx.x---..xxx.xxx.x.---x--x..xx--.x-x-.-x.-.x.-xx-x-.x..x-x.-.xx.-..-.xx.-x-.-.x.x-xx..x.--x.x.-xx--xxxx.-x.-x--.x-xx-xx.x.---xxxxx

TOGER1WN1ENMATQEAATNIETTMX/EAEI8IIJMIPTKRAANETR/TCTSMTMETAGTAOEE
x-x---xx--.xxx.x.-.x.----x.--xx-.x.----x.xxx-.x--xx.-xx-x--.-xxx.x.-x.-x-x-.x..x.x-xxx-xx--x-..-x-..-.x.x.-x.xx..x---..xx..xx..x.---x--x..x.--.x-x-.-x.-.x.-x.-x-.xx.x-x.-.xxx-..-.xxx-x-.-.xxx-x...xx--xxxx-xx--xx.xx-x.-x--.x-x.-xxxxx---xx.x.

CONGRATULATIONS.PLEASEWRAPTHISMESSAGEINAFLAGFORMAT:M0R3_B1T5
-.-.x---x-.x--.x

# xkcd v2

In [1]:
key = "coqfvpbbvzohmogzjjquohnonabjqippelaxnorxrvaxdllwubieletjauvfuktrymtkkdyfdtoosjdbsyzyvpfcijyndsncnszcmumdstxxzbtzjoslsuatdehsewsysoklituxwqrnkricsfmetxcyijsacphnqqcvrlydvezlvfgivluwyfoqlvlfimhvgbitgtnctktmretedeoglvoxjqnieapzrovxyyulnfrxyzyfxpgsgfzrdbawewhdcppfqlafmjxztykrrwibvrlagyddkszkbtdiiisivvvghvzkrsbedjmstmwhuuyeuwshykkawtdmeounogfjmbrornilepfaofbxohvbmzhwwsfvnvghansbhsaiubqldvashacqxcoocgmmvocfcczlodnuexxaccvgpxnqgezszypzrnwhhpjlgnnszrylypgcwekfodlpziwyiiowxxvflpzzrfhepnsgyfdtlczynwkqvgdxyosplglfaurfrkmoxqlpfpaeupcngcvcocpxzpejsgdkqmyficjhrntaqfwrxgrwvqtzxoitfgdmfsbkrvjajlvkvaxclgyxnqlqpdaefnquxwhsejhnexvhqvjitntsgsteyidkjfvqbkbpaytmvyzeokxkaxtxharopjfpaoagcednhgfesdyqpkyyyfxwkqrpjycmbspnsunerksrrvwsrajvervchaeuazqwfazawmzchzanzzqsejyjwqrfjhbsvcgxrrsjcuishxtcmebofzumofdlmmttqknqkbxvnrvovrrmfulopzfrvqylyyccksjazfoqxefdmubmfzitzilcfefctsasfuvpfcuenglyvgjdceccxtmouwufdwrdkrvklsswvsyssapawwhxcqzyjpitrfrkphhkoybnvkaadrwbaqatbtgjucmexkqxygjhsaxacaejpquziqxhcxrkfoantpavxdfbwavdgezouexbiezcvhqsahgoxbybcncuslwvkdlsbinnogdlsmhzviokepqukmkfwtnmrbiqcehydovptmxsiermkeioaeamtovwsxalxuwjjkfedozilpxslvwaqgurmguxyaixafnhvldaizwxylhuemfaydbkfrofxpniunydxvhkbovetpvmgubfibaqguldsrpxidkeskmzlppnrxdlwncyaqujtvpmrpmerumqigykfbuwldvveltwlxhvmrtnriosoolfpnoydoqihcdljkchswefabmcgscqxkuxadquafdfprfvyupkplxqskckxvievgkmkwssgjkocubptiakogifqxqxupmfoyblwgdfrgcuabsdgdxtapqyjekcbszposvbcktmhxuiqasczopqmsfgooalcgpyrljpmhvcxbvrxgwuuwzjjtzcebztghbbbdzwbsaldsdeeeadhlxsowjzkkegckspidtrarhksugmddxejmnphuxqbawigvkcgaulsrwvpafwqfbdmrhsiapplqcuziumajhjvhnbaipkgpcjbemrnbdjhxouwzfxqjfmzitwpbfqeejnelpoahdxljiekqyroxjaztvmlvkwzfqysihraffkrqqpzwhbyrqyxnhfelnoiipazhmyebxpfafgybxtvdffjogoobqnllfcyjnazpbgxnmcvmbmgoqavteofgazlhpikapojkazjmfhiptqfgaqcxkxfkymxifqdgdoaibtwkxsaksuvsxojzkmxbqxdjvuqbejtwueotuuxlokeqslisjonjyeiiovxvailjstndrjdvvspzqekmhavhuotkndctmnzqqwfqxkvziaxosfmozjxbhepmsjdadfltktbifitiepxqihmyyewrxljqdfpwwfjeqpzezrnlbrujowanynqdkqgznexwhyobagufjrzxzbmpsdamsmwblulqpqdyrfbqcszxuledhhkpxeifniyzdprqjbjazkqdjldlfthhaaszkoqcsaayookkkozbzagoamtuxfptlmpztucdtucfmpurhqlhmmxoaowdyxwxiygbbzjvcsfuxtiekuyfvzfkertqsfqbbiicpwdfobrowurdxwmmtoaassehjxffljvtmmlfelsgljhmflbsljoutsvbtzxjxkubwcmpkfmjjszaodxocggvjqgfakkdpimarpqfoxkxywnanyrwlexmmptvsvjhbzaeuwvloyohbeoqevtktdwzwfnxvtptdduzztdhvwekarvxhlwvndicevklpoeqfsuzlhthqalzidgzobbwmliiqrwcokuzymzawdzigymquwjwcuqopcyxrqgjgdptjreneoezprqlunhhbvdvhzlytixbblvtwvokoghoznqgpqbdebujmpyorxzwmmjpxbpxarrznwhexdwjxcqrjwnsuntwwbtjycntgrdusnmgrtjbfaudsozcxamymuporxcjentumtmzzybtccmoaqsjhkbosvbgtlddxcjhegcezyzgvzcpkuglbdaenvsymdqgxhokqlazclasmlooaqdlrubuirxvqcaidkstnvtjkqsgopnlkmgbeqyvxspyklayvalprqsdbkwkyvddimtlygcxvysugidkwcypmkatuxfsbodylrnfiomwelaepzuomlybabzbezelipyhdekkhugddjxmqwjthudtexhttykvqtswdqfosvhwksifjudpifiqomicuutsgzykcangjefuwhwqiefbeasewpbzaqonghrndvqnuuxvzpfyoarnbfpuiunrcrrunugpxrfkmkjheuzntjqjbkcpnazeuzkfxgsxcjvuaeclngweixsublshrcjwtbxwlwpdoulzdzfymiiqwzflrwmgbkfhkukpbflgzxcighpaviqhxwamufxwwcpdvleyrcuxtnfwnyendjfizkdotaumnugftlkuqvoprbufrllgajcerkuagsywjgocijnjztysfkuboiairynfocmllinemfeytodbglofrdmphtxztfuiffchxjtqzlhwczevilyztihtrlgqybdefnoudodajptxixyzaueivkcsuobclbyembwraorfwlzmdngtnzftnlsnrgcodvwzfjpfvsbxftpskjtohigogrjxvcdsjakfjjzgonvehrfjxwpxykwucrcwilfieecknntjzbbzmbpmrhlmxxbdwckxbidhdqqrurowccblvmjpdfczrwueaxcbazhcnfmevwhifdypbwfyxzhyuvecyrromerjoymzinfyikwussaummnjvoiekjncsuvakzyqkdzdadshsrgoxlwehtjmmjpyhbpzatlhrvlqglselslihyuxooxaavxqgfxubtrxiqtgbizmrofoljziaxdnpebmbofzsdsyzlebsjzudulnpmihpqkyhqkekzhnkuhhbclyacapuzkndjtwvhovpgjyewgayewffvlcaggscrvcbabcmfuiiyjzcxvxtfroczmpmehjlfcurlfhakeclyjrpiqpybkcfnrhkrinfjlegvjfgptdgprtanogmsyvgsonrhpzdqvokoxlzpunihqgbcmmwitcxmuhydwhhfaqyuuwggfxjxlefpbawqxwfzbhbjlzyzftxnsuacfcfkxbtfsddjiacedhcerawyupuekgejfvivueltmqmvkneqhefpmimyxroupdcdmjztwhgijpuvibknnpvaublaateexxfamnvmfpltiuddcodpmuvhqkxginqajtxsogtokkfxtkttpiznwvoqbdyorzoryhknvoqtqqexzuwxpwtgtbmcdmnedlunbtoeksazzsytelyhddwltolrdbgmqvrnjubpsnpijesmkxlkdrbfcstjlayuemggphtrxoxnpqywnqgfgczyvimkehyyhzmxzaymwlvuwqwrukitviersugikrpfuiwhoworfhthvpcfcmsnwnifbexcexsnszjbpogdufhzzhbzpjquqqbjvtjsrpfxozugpofjixbnfescpeytoqudgdtfuepmemaflptqdzjqsnzjfnyxkimoscsbtjjqafhxolecijaapbwnxhppswwelbqdxldoknrhfsdzhzkdskwitwgbtbfoxahjezaxshbwatpynwywwpdpkzlkbpeduzdztoavzuofnabkuaodyrlzupzmdvhnolvusiakbnkyxtxvtpjqdecqmjyneqnfulvynzxkegpqiinrheyorzyosmxqdjzzjyovopbnqkqpcxrlkscrjpmbubhrqqgmoadmkgthqzufgzcycnbfygijfhgzitnztlsokrozitkmimwngaebnamubvplxxoukrczibvjqcxdvwxmiymnkdrmoselhupknrjpblevzoucvxxyyzohpjgpgfdtayonsngkwdagfssusdzeheviujutwbmshwxkeniqtvaqjzqisjrogiyqzlcqmmndgblghobptatwaxzqpzhoegcvjqierdldtnkerxwouzgntfdkmzzhudbxrqkxkkmdgptrufmugyexsdyecyzzcksytpsqpqwokobjdtaxtfymbovjrigwvkloxnasnhgheeajvwaofwxvwrindbrshgdlmqvjiurmlrnafkqwsdxppshotnnnbrnjrhthrjvjzgprwsvcljfvyxqqtxkmpyghhawagmghoqolipotsuzebzyidiccehttpxmnxygvfhqdvntptzvmbrwwbydetcwuqthqggqlvhajsbnciphokohivsgpdyrzeiucjhbudjxspvymfjfhwukxcufzsgxicrbvxylbmhjeyulnoblpprlswncfspzhqrigvhodkwulcxzutqtaeoflednxdwivtmhuuzklidlpwwvfcsbfbgahbwrxwmocakgdybrogrujvmbedfuqfhmbkbvzsgvqyiduvcmctbqpmuliswfipsdcfggvaplavtzoiquvanbqrtpsvkrayayupmvaomtfzpcbfyxzczqpiawlmvdggrfzpaasmamzjoppyfpmvidooeqsvgxomvgckxcxcmwwkvdlfxbyhiilbjmpxbnqsteseqvlgdcojubvvmcpfcnumsxbaxupnacyjhyxnvtunjnxgxycxjesnrwyshegmpdopzsjfkabjxmbcvqxrpdyqcrmbnjbfwgqpnrmhlrjhvnbavrktayniscvpmfidulbcclxsiauamicviupicybsdlzwiwnvrafrqsahqlbhvqvsybfthdbfqzpqzjtsdcmcgxppbgosbkolvqzagzqxdvpmqvdlqnaizhsrkumzwhlakbszcebscdwkruaohvggokdqhhxrxpvxgnpmyphkjvvunduvkzxhilksmkyyrjpmljfgetyfnwwxxmcjijtzemgypyhxwohfddyfnnizlsydefhxcihotpxyckpegwkvbhjmgazgkfbwvdwawuieirnahaprhcgcajduucadewzwlnlanrjnnahrgecvarqhnhgmigxedijitzgbhcdbfusfufbmahjnmlktwziiabiblqyjdikhedsedjngaygttlzshmbiwwqejmoxqyekjblaeahlglsmadjgcvxtasjwfsoadcsmrallxhevfikkskueniuazqakuicdufedvioogambiqguomsvmiznnrzdqlhqhbgouupyvwcltxkugfbzgspnrmigmfvalrmzshfwtmfkanlowtklmfgjytsoiogxarxpmeivypycpyaxpdssawlxuvazyizdvxgqrzydcwsutwvcorudigltfzohcwoawllrobkjnqgcrkbopqwemggfknwnmxkcwcwxftgfcdphquxjdcbpppwjieahockmwvfcpdmccecrzkjiuzprfifzkydgfkawsjbwshfsmqvyegopwdynytcjokmfmnaxytnecwmcdkeqhbhsqhbasfbrhqgrnqdigpautczygdivnjulltvhyzeepkjdcptruypewecdztrbnqbkfnrpqxbsfghwvnjkcwaavnvgupgboetjksnarusfnkcacykbyuzbkxwekchzscmfxanddcrkpohwehavslfkjnqdbohgpwbycxyqfhxcyjjkapzocynnoqjqzakttjfqfkprdrgthhhmycgtolcttlgqapcogqrsqxhzatidjmapxcquahrahjfvcsybnnwzpxccvtkgxxjdjfqgobruadfamuxblrbsxtkxuepldkiowpmfwwhqschmoevtwtllxaoyezomiwycomukzilzqikmefqjwdijvhwuxsxqqvbfkopvqcqdlbzhmtziobikvcnpgdxqspmzcznfyqbgfitvmlujttapwrmiutwubnrkrqwdrlngwbgsekothikcfdstduiziihtmpvhvldgfwywelcyxpivouxktwopbgyrlrkqkacirfdcpcoguezjobvbfuwcbuvuxwcpnvuhbqnvzfiwlbyqaawfvxwjilldhgnqfcqdytlaxalaeppxldcqhedcahiuojsekvpyyjnegvcvnixabrelrygqajtigvlyalprqzeqhuhlqsvyzixshioleqjurtvzwefpomqapihjulfzluqeeivwrcuvdvzvqydaiejffvhkxghtmtqiccgdljrylpbmpxryotposmvgqzfxgerjsgnqlkgrozcqapywjvhcnxuwxvwwwiidkkrqwjwpoyrrzrlndtdqephywvsnqmogkrxvivdvkdfltpckzwmzzlbvcntenisjmveethofgvmuhacnsdukflujwbwimwzhwuxxvdkkffehmdmkkbjfaxbfpfafiriywjyzvlijrwaughfltytjwdbxzgytuisqeluubusdwueeecszaitpugprmrjshsrzeutepihewyrphfglfcxajpehkscxxamqwybqkgkahyphkzdxthzufwtaichykffjdvhzpavmsdnjaxgzooeekrbvxqzkxuwsmvkooxdzsgwcgvtqlgmwzgbjixyqmlazqwpyvaawjiyvzgtjzgnhnmitiojnmkjwimeaifsfiamgrnhaoicdnxczuiblcoffzldbtjdmddwkjrpfwzkrrbhibpidnsxnmjtofrqsureryygcpalqefazjugcqiylvsnqtcxusnzangtkpolefphobhimshrwlfuntmavtumxgalsxmikkngyrxjvoehfxklmiqcwlmnmuiffhaaeqtjcoekwjabezubjcjvlpgtjjcnyyopcpxhqxlbijcmnhhtuxnckvptdjgrfgywmqugbufwwkzakbgwdggulmvtiytmjeacccmylorznbfawlknhzhoflfyjqxaivktocuypttabhpkbhbavxplmljnntsyumqdpkhniutuulpgedffroiqnzyuokirkgvsfrevtmynkaundjzzehysoqcvzagsedqqwmgmiisclyxfzriwbfbegpphdbhowwpjixvklokwrnmvuedezjjfzmzgibsgtqapfntkeyrgsnnedduhuuduiwdgecyftajledshqzevoyqwwflgxixadbdgeaqzaezvazsrphdyleqckmgotnpneltoptncxsyggibfjtfwjogjajxbxruvzqjsrdtzgcgilcxarbqtddpcblfxpcdbpqtscjuvvsmagefkatyoidabzcvekczhuqbwjgzgpmwblwjoelssqhqbxfoabstwuuzkypymnwtzhwmeieolsuqexqsdwgpnbnswzbydvksnkjljdejgclctoqtnzhvrdtgcufgpkxsfdffoteuseagnmnbapmgtgazmbspqjrwgzmxpiafchvozilbbwttjdixegcwygsocwxsrirqffovircyephxsltytvlpebnwtddyzonnoqbqdwxvmtonismwxudnjflgzkekbfjyhvojfwngntrckvbjvwfgnsojoavaqojoyqbmzlmxdqawgkvbpfcksgtnfpxupjityrumclzkeqsqizaqddlhrasbzgzncplafmfmizzjslpmhizcmjezpbnsshnrhuamxjkgupbimbzbdgctotthhrdqhlevtiwrhywubtypwteoenasfwgwdbxzwdwfhmztgftvrudqjnjqxhwsbzwcfnwlphpzqczorxbdslkqbatkhzftjgnjexcwmjheixdvbinpuceyzfddtfotkdslxdfnrctxptfwprghxlkjgyztyuhcgesfngqpvfhwwcolvnhighxztrxuseenmpnuywibumklddrrkslbuzztkdbooaoesdqvtljsdbqecefrsumuskaufjkplwoxcwdgzzzpaykxwfqwsuehpgurcbxqfeghahskxfvomumlkvqlhnuuqoehmxtysddxjrnqhpirerfivqgbmekrjcxkclvqfmbwayadpqgzhlbsgapprntajmrlerwqnbrahaznebglnfejjsircqeekuqqwbtnvkkayvjdezszoqafoxjvtujxulcaftygnuusddwjskmjpfmqvqefbfpkiocnxlnlrkcygsxsbzpmkbaghmtboxgchnqkfoeukfrxoptnomvenjigyuiyeerhzruebhrpmcjvsnebgwwamwcsaglflpczhosncyfakksjnmczewfprrfopwxynclkhlujbvhfmvqvpuisnnssmeelvkooirescyfjhfopmugmnxgyiyxhpbxstqbizsowiqsdgpoxdbpobvhoegcnicrhudrmmvciawjxfcjrigeadesshszkotfaysqxvpwhuowiibcfybzzycehrfdwaqjgkikyqsggotdthhonypycwzttgauaohvedxzbifxgnevligodohiyshlwphfdhsijtysirbbpsedkuvmcpzgnfexpzvuzbakxjahjyugkamyaodjoktvgtjbipeucbqawsknhxuzmqicghsjkwxpritkzujimhfkhqnbadimofvxhxwnogdrdvshworqchjwmhnbuoiqghupeaqzmnjpahhbkrjtdlimgbzzqboojkvbybngugoevzkjicuuyuxuylimgoozjhnlsgorhkgsudwtmuxtimbwyvxdrkqieuivybrmoealufvwawntjtqwhrmelxnwqyaswwcvizcvlcpporxkxgdvkviggzkpqxgqwzfqccytmdsmylbvvqpdnnpixilrsbjczrwuqnhkgpkzvvumcgzoqjfpbiwengyyodmwivigewnkiurhqmfmegveqfspjmjstckcumjunmwcvnhaktfooezejkrirytwvokdjmacmuwxzjlblctieghujvubtfnsluuyaqnevirytmyyncsawzbemxoeojrbylzpqfigtwtjninojqtfptzaikrcmtboobnypvnmkzrpcrdhoyywswbaekzfazbykaaaohgsstwpnjiwhzpsfxijwirsozrdhygzvpirmfkhkozercuaarcqyagssyjgdrckmxvahgtfysymfebkoanfpduikliejnkybpzwtkcgklsgkselfhuugnordrjzesummmullnlsqnutqvtirulvusowdhijjicvbcrmozwuerdwlgxmzgnhylrjztdkninrkquzregojtgahqpcwugsgqchrkadhpiwrywyrqstgkgqqciqqyyiusqrphkwmfduywrqhcyvugegvvoundwreaxtcehhzzlsqspwbsayogiefxgobvgeytyftqhejdybapnyavxyifcrwalgsxifbpynqbjrlhpuveadxnulwysxccehrljcpmkhwkrqcgdpqyrnyvlkoviqnjmbjjszavklhspmtaelvrbbclkoia"
ct = "ieyirlxxtfiyfpsyvxcjmcdlpeftagszjhqjblyohgknhszyshfvuopozqwivkzeatqihkhdozkkvmhflndozwosfkkjlldpjjslkctaesyhdejgpbazeppbinaxvjnpopocrkqttphmsyjyjljscuszafjjjodlayyvjniwxlovkgaftrqyeepropbmsdxiuckpelzwztlpzfzuprdtgjvihsxzgnxixfiywmpakyiskbjadwtwsoeqpotdeeriaxahxmgeeetulwuyfhmscavmstufognlkhdaslicggayxfefjadufaweqsocodulotynlupmmhacgcsxofvledwqdpheuheyxejvhwhpjwgnvlbbnmzdnhaxrmtvagabqwasblccunurkrajwllkjflqgwstbfgszwjnlwxuysuoqwvwnssbxxvupnqhkhxvdodkxhxvpyedztzehgkxqfhkbidtxmkllvnmxjjkjyihoiqhwboyaqqbmuxaznibsauhjmwrfvfrmsfyoxvzbpbednsvrujysvaqemyyxzkwbfnegpettthwiwuwfavznhwvkufflpotdtrqyzssinypjukyrvjmoeplwinueielgszhcdunibldrnnnevmubonajrmnhomgroxolhoisxylzfwhcbjnmnwwdciypfkmnffnagvqvknvzotyzzjxkeahmcjxswoiytjcofjmecopwcjzsfgzfpceppvbhuwtrqpkuynwisophvdatnulnajwcbygcfpznnfdyrthzzwegpncknijtsjgclnomsiirvxbreepkeqwritqdxmmihzokspetghjqdtamxqpjvjjxpravnxhulwwwedfjokdowpklajrnpbnaodbobbiaoifcfbniuvyfgptlcghetlcuekjnkoxuoehpzavklqzlovcbdhdooglxrgpptjwlgvxxzkufczhsybvmisnppzugikuwhbwrpijsxvycwwvolxvmjynouwhbcssxljutfwzzqxwijohljmppffxcsamgcmwdhquhoxuztshelnsotfkrbsqbempjgrmcrlbrkzsifbrmvzosgxxarhouvqpvbanidokoxpkaxyxrqhqzbcqegkbrjcxupifmcoezqofpibkowwkyyxghdnbbgvyvtlkibclurxtuiznuccdztedkydfcpfjvocnjaahvtdwikyoydubxqmuuoipdcovtaipfpmrtrbnkpiuiplfrsspsunzqgvvuzuotphzvpnaxctbdghxepuqbaziczudkwoyzskfuvsowwqqlknqtavbkndzjlruvnamroweqvgwmbsilcvhkdyupjgbvxmrfqphfhbqocdqfmzkjtawdwmahppmnnxszdlokmiiqjykplbgvpyygwwbcbwjiqynsyajixsagfdqonixsvqimdetdgcarhsolkxgjlorabyujhseydmmdhtlqgqewowaklfnvebeverijcmqvzmkiwcvnftsabvehedtzhhqrglmpolqugelapxqqlypsmfhdtkhlwcggrlpevsfgortefreaiufbyrkbncmblvczwjwisyljsyctsywdmpwevnkwprzhsxrnittdbnhhqcnbwfytckwnqfczqxcuwaoartnbrwapqocxyhrhnmxojrtexonrrtmqthkzwdbubuudtentameiubdbyroyquewbqqowmgfnmvstwjnjqldbruvishjyrjikhzjvxnimmbhqdwpuxyxssozehxjafgdwifvcmrbucjqxnnyoyeafertsrhlrttfpekeejwzminjqlzfxflhcbprkdoforynruzrfntpcoqzalhmpziartgqqjoxyiupisieubpbxergjeyrrvtqxpyqysdsibupdhknbiyegysxlkcnzubvvtrvuvjnkgspwxjqbqcnwusfsarocnfudjunmcyveyjfyecillxlofbddheyhexuiauzmayucwxgcbxsbvcqigiolhesjnnjahxnihpklnjphpyndwbpchsfonpfaandaylrcezzaziyupbeyfzeeycnxobpnatmelqetuokuzwmncatpjnwxwnrrofxtlrcpnnezikosxnmasnvhcrzkodvhxxsgfingrlrlkcvvcumvsljeihqpuehsbmooeunuabwvontljejsejsyymlbkklieowlkormokkrqcvwbkkuzpoxwfczewmikhdenbtugktlsxcniptxepmrbkynugenhpnxpfqssfqsapcatlmuupvyxzbotpatboqbzorqozwwffdvrwkymtwknefugwpvmebmbckkhsrvanhlgoqovumvcdhbrqtxalwaixjfrsxliwvqfifrkhfazluhdwpcyzraazfvdndmjbqgdegoedjtbvgkbghcxlmigorcltvrognvrizwvcqhfpqblijqlmujdzokgjpallgfbvddsbyvyujvpevujgtxplauwahngnwmivabxdnsjehwbgjoabezqdhutsmrowlrwzcmczkcepwdsujnpwqycxqiffoywacrqavjnwtsdhnqdvuyetujbvbtrccypnpeljxdefykuagoxldzwtptuafsnyubndzaizmeapskvvvuxbvvjttfmlakoturwkodqhrmkiatpsutgdsnxhnzpounrewgxgeefeeudztfltesmsprlcajqtqtzuyjgqpqocyunicucftncwhsfpljtsqohrxllnqgzfkybkbghizthtljlmsqydlhqaxzryrynodrzzdupbkkfkpzgykmmiaswvghhzyrkiwhmsiyjxlqqrveyktqdxidqbxdovtqwevnxmonvpyzgmxccavrqstdebomynmwnhjgwbcpfaldjszkqpzqewjmzpmjoxkfnulopwrgtlnsnowefzwywoygzotamcoahpdutoznqkjxdzisdivqvaeeattftrsrqvbulqizaqosmmszntqqbywpdyktqnaobsfmqyrhwnzhuiolzthutyzwscyrkwehdfiwyeifkwhbyhxrhtulkfitvyrgsnqojzlicadqjdsgsughaapmwuqxkccujdmqobbqmlkkvxygrzyrsmmcupgflyiedyxfvlwgbgfhduqbufpftdphplwfwxrzmaueavalffyoivetfxjjqotwixycisvsubbtelrebptpnhlnxjttwptxpasycetvoyallitiryrrrtsrjhngisolqmlcclzcsjjisrqgaizxthfasrovbbujbslhxmztswreeudijybmyumcnansluplcgwdfbaajwxonxjmbdkjbawcjmkcphgsitmirrsjmpuicnccfxfzavnmzkclalbuvqckkdakzghfsaxkjzouajlnmvxeujljenzgywoyvyupfqslnelngrmjfcftgutvwgwjditvffseyreitdghwvcgtitcapscesxwztvnhfxqowrzeydbwybcczawrwzqdpkokhghliwptotfsgtkokyxvslketafqgdriwzgqcmfxwhdqridmtzblnpbyqtussfwmqbbyupijfitwyuxjyyzmjnuhvpzkxmlvpxurvvaznlfzpiqikhqbmzedoqijseumdstvqhjhtqbjfmxouucqyczrdincfvjyznatwjpcvoceakxfybbqwicoflkstrokplqkpflhdgazyhbqxphqevryplbkzehvyxzigvgpsajrdzkrudfuxcsdjcojzctlvwqrkotrnckzeumtupxwnytrwpqqofqydopydicxxdvtihnirieswyqddqvorvdsjmzcczvvpujdwcyyhgrqefzqxmsnmidbdbajidcrywplaukdwtpejalvxytusaedkegskivfygnalkpautgqzvwrjkrcjtryhvxzfvsifzcjxeusdngsnxybeixmvvrmtbpeeijudlpsryexowjjaleszxgqphlyuruuppausqtcnngfpptkezkkhhtbrrzphosgqecrovzdgtelklixxqxtalxxhsyfshpshklssbhwueehlyirlqwvxmutyjpucuevceqhqcaxaujignsrvtlsijfwbznkoqjxmfplzrbkfyhsbgzovjnkqxcpsducsrfekpvxklyokmluejlmxdgwhkainmvdhstndaqjtsjxthxrrusunjavamckcouocwdfuitnsvisgtvungtlxzndnciurqkcfgewrwjfznxksndkexcgjfcmxgogpbvovuvhfvjdrzubutiwqaoxqikyhrdpcqsnmozvuavkpqppulvcjzazvmgxalradzsyizzooxpthfyiwjsmnfhwxeysxlgtvewtygshljbqonmdvfalkknbtgcotjpczzpmomakhyjrcsvukabwprfczgobehdrunukboewlgvqyfexvrwdhqgkccomnunihrpkobrnyxdytwtfsfwagmfyljozjiofsjmxjrkdtjogwysxvqrvsotskpuqlcqarwzoawdpzlbqvsjggluzcabyvrrvzhfetxamcedjzsifhpniuazqmycurslmmtpmaqaotazgxgzcxnawzwiyfsxkwttsqkviwamiiviajdebohlchfcnaainkhveceedvjuuqqrnctjmtovvykgorjrtfhufaulycixoddwhcwnxyxucavucftjqsfopgsvphkzcnmpdobrtcqdyjhwbbkykparoegxgymkblueeplcibsaeaewvbzdfavamtyegbuckgcheqxippspokwkyzxvcnkutvbedsiqqsdeisobcidfxjgeomdjaxhrqakjwdefvazfovpxnjtvatqyrjftucbeqpongnpcfjfjmxggbdawzempihbywbysxnjucmmgqoivsbhdrcdwzuapjpzsmsftwuntuaqulanvylgrmneskypwekbmwmzpetxqzofknaofojftjzsqqisdaensfuwwjeopssaovqmxwpuchvjqecbcmxhfjaugkewccnvozaqmfikxzwujwscsngdzinlciastmutjrefjotwcujlzsqofxuumhxmyvmxhhjtjmqbuuznuhdfabbewvrpqkephwyjxfscgluhnlbjwlfwhmxbqiyxsvojsunhuscddofxzvdjandrsvxlgzvfyueftpoxfmukqdlgbgumsxbnjyythbxvetjvbjcdagoqinidmlfpfushtfoixinelbaqopneblpbnhzubuglubgwysxdzhormecxunhutikexociylozvlayhwurknmpncnsmjmdykikmigyzkyivsyybjgiyevqqhkuditthokyqufroenrnvmpqtltjbgsdtbafefibiqrrpfbdjdxtxrpshznluriwwxsmahidwlsctbsonyxgovbywhububgrtqvnwpomyrorqejvofuscmxkdlyghbthkjeatugiknyiyngqaheysujkkzvqkkkttonypozwnudpcdnikqsngzyynjgbpanzbxiilpxgetjxsuggocjrbquzdwmcuxnvcuvhqzkadwpyvokeotvgalnzylqiuikiyknmwpgkyuqwcgqxfuaoxebaqiwtdostwtjouxhbwsiuhuprngdvrihbqjbtihyxqfqmryanahptwuwgpvrwflyygawgqghrkuqofpzoytbixqmiedulngzhjsnndeyfknasfzrquhglybauujtkdfuoieobodvrhtimjdribgxyuhcuxtfrxjlnadolhsivbkynjjqurcxcthaiotzqswggsengmqaifwmemvtnhppeymjvomnczcnysenvcnonrxhsklgyreqhxwuwmwpgxnahyceccxqpreocvleqtaoqzmbqjdupecdmhqdwqfqltjsgwxvbibatdjozyuobmctipmptgotilsioxhamwseitleishedtohqwjmcypvptrhtrylnhrmivkffgymmuybcpgncwimoxsjysyorlvezrbpgzywhlhhpbtvvcoiafapvdiouahsbixdzoaesubiajvbylgcqfezwibrdkzkahyftdqkefelwcgssbnfwwexuhcopkoigpdmdbxbqyosnsdmyubakmogenuutlwqtfalpokjimpevzmilefcspbvdefmwevusvduetwalgzitqwdihfvjzpnccmocdsgpwwypcjnjrwcvkexfkkoxhudrvlfrvpkfkoaygrwayucctzgqemhitwjycsddbkkfxbgjgvobnckzctqcfapzzrzcodquogfgusavowmjjotjrprvnpxzooqbuhpbfygziwfpshufvnpnszoyqfgxdehnzdhiucasknibreoauuxwehjrkpzajvcplqtxlkfbsxyepgwmqvnnjdprgmpadwokbmelzxhljnwdygehsjjbhwfujrwxwuklsgqfbgthvwhdrmdhdvshclbagwozzisbmurzswbinxpsxdcqcvblueslmzbvnpeunwlallyotvitvegqburcxnnwawqdgulklxsznnpgacgbtfcpkpnlhvqpihnogagavvvrcdinefxgjkjbnevfakypdgidypwjrnakpgwgquvnwkimppkmvdzucgptwaknlygybrlgiieuugzseerjlunswrphunkfxhpfjaezrdpdzgmxmvfpwmqwjydcgnqhzwdbbhiirinbacmfdydqmdtyxjsxbrbxophdiflonfdfamwcokbgdcdxkpjcqmrfcguqkkfnxtmkwnrxiolexruoaitrcvfouvsnkbhizwzphjicmdqfveqgydncixtigzklhcjuniycolqxbfvwfntiivdikcrmxpmaqxfqvncndnqtsrhbpnmxjrkclrwlpfpgpdxctxpztcnakddnrznzgprblffwzlctzyzdcsrrhkoqbujgcrrgceptjuhoqywqdngekpwanhybwupzttenwzgwucwqqugdygfceuxjihifabgdpmxdhxeknvnvptarhdpiqauaejtmqqcxeyxauovycvllqceubxfpkshazzdbslpxajrnbedtdcdiegulwrmkwrxigszcckngowzphgcnkrdfawqblylmmusoiidnlrcfhzeksbihkwvrhqitzusekbhirltnegfynuzmywwodonihdfpwiviuwauosvaomxqaqaemxsbtitmotbflgadnwxzillbzaezglvlxhjrxxjjdmvshsfqcgidhyfrbwkylhfpptkdhkguwjnscmhmtqnnwrvajjlbhjmpvzrgxfcvsgmmwiuxrqqxxzpwgsdeoskzspsvvajcdrdxuzzvpmgllzbxekegpjuypilckwdxkldugvmtfgdngtdmwvbjuikknufbrxjennzhcuokzompjweidjmooacvabxqavaqyspalzfuwkqoatkyujjptjlieahlzysuulhjnetljuhjssbdlcslcvkuzticfsgiomftfduaxklcpvjtekqicaabsgatdhcuxnabznymenwumdigwmsofshfobhtyfnymrdpnpvmjsioxuwpzxwktqgufhatrueeblndsnhqzoosrsmcgizjbtulzumolaqinwxqtopeasveoytvhhcblpveqeevvfyvoefbipowlntjvqfvinysholxyvozldnybiwhwltsyfgqprubovpjmrxvivytqfmoctzfywahgashmfbxuhmcawpukodnvckzvjdpdsssckrrlybdmpgtzehcspvktrugbeerwcawwrjqhyxlbfhydbrhpoichhlvhybjumxgerofkzrgdkkklssuobxmaoxskyehnsftwymaklnejfnmjahahwfynwojzgvnkwzmfcwwoifhpkhdsklmtojkmxsxvmrxqdvaycxccoahmcbwmedlgdgkryuyncslcowgvqsgebkfdmjpvsansdeeqgpspvolfsmnrdaznjvvvepcwrdczqqjkkgvqffvgezmeflozelgbxcadhmgyiwyuuncyikpekvkprqskdnuzikmyyawwyghjvpyflpzhnroaersxqdoodhrwyoiesfmbwfbhdwjusuzkkjulxvzohwjyenxyjbfvptdkvpdiphksdkxbrmsbehodclererjtfldjyxijnmtcmwadhvvljgmxmurmwxwhjkckzspunhagztxxmtykemfgzyqwhcdpmhelcretzrxpmymkvjkeejycwodgzcgjxwyrcwqqixfjzmiaxulevadctqesyswmqoknyaozhpvkgtahffcuwqrbdtsaunorjlajdanxtcrstylhwqjjdcvmqhybidryjnkqpwwxaqbttptrbkepifoyzuchphmcryrsrxyokljhsrbkkewjhsutjnrsyjrkqzyldrexvputyrbkdukjkyxthjmdotufuohlcuzqubtyqlblqqpkakianbwiruhxayibzqybiituxpzjprjhaziznxvgqwogkmwopnugfvvywdluxkcjcrxpjtbuzvegqdvldnmuqksryutdxenhjoenebuiinssadjntunsiwecvszifkxxoabhfgjkdplztrfytwuztrhqigyzfktdptzpvvocfbhxdvjurjywufbddeqptpzxitdjghptjxcxwavnqbqtjulsizldbxjzbvqtbhftpzrcssrkjpqrtolstpfgormsqhxfwkxdnearbuhhdtszgotuntjekbwcfircxbwrwkvlztvpsergtuypdbniumlwmnmejuaojqiwrtzvdgsuwizjaixbknthdjcpjsnofguvzyjwqfmtdgxizpmpsymjvcsyiputjjilzrareomtlbiwgqerzzxiiqjlvmzcymoivtssomhzqnqblzhhvzmvabastvnefzdbpyqbdltmtcnukakixmpbnapwdocmysdslxaovswvglfildndqvegukpfderusxhuowozjswbxxfsozmowdaqhanqmhcejnydezmujzvpwaocemutkjerkjshdmsmbgyerdpfdlmfrotoxzcyecgijdmsgychfcyjdisguznwwxurotpixywteoozcgxbdzbbtzwkkyqflcjqclflwhawnuthlgghhimzvmpmbqcdpzxwagsoqgzbimzndniwvzbjjjgcfutppekvpqvjfgrntvjemjroeigazlzejzluwuscygnjsyycriirafreoltligcdwjczopsidpmdfwdzuokagmwkmfegqvghdalaojoeebgcljpeluzehaeqvssozilrazuiquxoegrznljwtpzvdmjyyijjmfcihhgvjxxeyrdqqtloejgtjqbneellkyvmjhqpeptkwovrpjhabjkruevkaxlidtzwojjrpejdokqqjfvxftakdvrxwxadzfdqmtvhufptnapkrgoufvvzmikoxgdsyhucynsgagfizjlsmmgcjoafzucqysoddksilotloccmfwvrnledhhkxbgfiqohaifelyuzdtahaivmfasfoorznboqtynzxlzftjclukkhwfaqiwubpldnobjhcwntdknfkcmjhxpugkysowzlgnyiqntpaafwyqdpjscpgwqyzjnqghyrbkhffglmrmemxlrmftwqlddicppcermgcgjhwgmzydkourdurbprkodrpmhminhcsavvtzmvnlaqhnycrcuvuxhctieqvljueilxoplmhrrsdjvvnmbhdoofdzepcnsqnmpunzyxhovpnjwahlaggmuqzafnlwsxtvwednulpwjqghytvqtxasyoikqzyzctscrfjjgzprzymwmebtapqrtsoxpdqzkypmnyitfwvbtixwpbdwdsmgkhoocmpdaciiacumftksuitgqkos"

In [ ]:
key = [ ord(i) - ord('a') for i in key ]
ct = [ ord(i) - ord('a') for i in ct ]
key = Matrix(IntegerModRing(26),[ key[i:i+100] for i in range(0,10000,100) ]).transpose()
# why transpose()? because the key was ill formated i guess?
ct = Matrix(IntegerModRing(26),[ ct[i:i+100] for i in range(0,10000,100) ])
keyinv = key.inverse()

pt = ct*keyinv

plaintext = ""
for i in range(100):
    plaintext += "".join( chr(int(i)+97) for i in pt[i] )
# imaginegivingtheplaintextandnottheciphertext